## BFGS Algorithm

In [1]:
# author: Bartosz Chaber
function BFGS( f, ∇f, Θ::Array{Float64,1}, Q::Array{Float64,2} )

    g::Vector{Float64} = ∇f(Θ)
    
    # Looking for the best step size
    d::Vector{Float64}  = -Q*g    # Direction
    ϕ  = α -> f(Θ + α*d)
    ϕ′ = α -> ∇f(Θ + α*d) ⋅ d
    
    # @show typeof(ϕ), typeof(ϕ′), typeof(d)
   
    α::Float64 = line_search( ϕ, ϕ′, d )  #  α is not a hyperparameter!
    
    # New weights
    Θ′ = Θ + α * d
    
    # Hessian Approximation Update
    g′ = ∇f( Θ′ )
    δ = Θ′ - Θ
    γ = g′ - g    
    Q′ = Q - (δ*γ'*Q + Q*γ*δ') / (δ'*γ) +
                 (1.0 + (γ'*Q*γ) / (δ'*γ)) *
                          (δ*δ') / (δ'*γ)
    # Q′ = round.(Q′, digits=6)
    return Θ′, Q′

end;

In [2]:
# author: Bartosz Chaber
function line_search( ϕ, ϕ′, d::Array{Float64,1} )::Float64
    a, b = bracket_minimum(ϕ)
#     @show typeof(a), typeof(b)
    x, y = golden_section_search(ϕ, a, b)
#     @show typeof(x), typeof(y)
    x/2 + y/2
    
end;

In [3]:
# author: the book
function bracket_minimum(f; x=0, s=1e-2, k=2.0)::Tuple{Float64,Float64}
    a, ya = x, f(x)
    b, yb = a + s, f(a + s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, f(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end;

In [4]:
import Base.MathConstants: φ
# author: the book
function golden_section_search( f, a::Float64, b::Float64; n=50 )::Tuple{Float64,Float64}
    ρ = φ-1
    d = ρ * b + (1 - ρ)*a
    yd = f(d)
    for i = 1 : n-1
        c = ρ*a + (1 - ρ)*b
        yc = f(c)
        if yc < yd
            b, d, yd = d, c, yc
        else
            a, b = b, c
        end
    end
    return a < b ? (a, b) : (b, a)
end;

## L-BFGS Algorithm

In [5]:
function LBFGS(f, ∇f, θ::Array{Float64,1}, m::Int64,
        δs::Array{Array{Float64, 1}, 1},
        γs::Array{Array{Float64, 1}, 1},
        qs::Array{Array{Float64, 1}, 1}) # author: Bartosz Chaber

    n, g::Vector{Float64} = length(δs), ∇f(θ)
    d::Vector{Float64} = -g
    if n > 0
        q=g
        for i in n:-1:1
            qs[i] = copy(q)
            q -= (δs[i]⋅q) / (γs[i]⋅δs[i]) * γs[i]
        end
        z = (γs[n] .* δs[n] .* q) / (γs[n]⋅γs[n])
        for i in 1:+1:n
            z += δs[i]*(δs[i]⋅qs[i]-γs[i]⋅z)/(γs[i]⋅δs[i])
        end
        d = -z;
    end
    φ =α-> f(θ+α*d); φ′=α->∇f(θ+α*d)⋅d
    α::Float64 = line_search(φ, φ′, d)
    θ′ = θ + α*d; g′ = ∇f(θ′)
    δ =θ′-θ;γ =g′-g
    push!(δs, δ); push!(γs, γ); push!(qs, zero(θ))
    while length(δs) > m
        popfirst!(δs); popfirst!(γs); popfirst!(qs)
    end
    return θ′, δs, γs, qs
end;